### StandAlone

In [1]:
from dataclasses import dataclass
@dataclass
class Message:
        content:str

In [2]:
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
class SimpleAgent(RoutedAgent):
    def __init__(self):
        super().__init__("SimpleAgent")
    @message_handler
    async def on_my_message(self, message: Message, ctx)->Message:
        return Message(content="This is sample Agent")

In [3]:
from autogen_core import SingleThreadedAgentRuntime
runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime,"SimpleAgent", lambda:SimpleAgent())

AgentType(type='SimpleAgent')

In [4]:
runtime.start()

In [5]:
agent_id = AgentId("SimpleAgent", "default")
response = await runtime.send_message(Message(content="Test Message"), agent_id)
print(response)

Message(content='This is sample Agent')


In [6]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import ModelInfo
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.agents import AssistantAgent

class LLMAgent(RoutedAgent):
    def __init__(self)->None:
        super().__init__("LLMAgent")
        model_client = model_client = OpenAIChatCompletionClient(model="xiaomi/mimo-v2-flash:free", base_url="https://openrouter.ai/api/v1",
                                          model_info=ModelInfo(vision=False, function_calling=True, json_output=False, family="unknown", structured_output=False))
        self._delegate = AssistantAgent("LLMAgent", model_client=model_client)
    @message_handler
    async def on_message(self, message:Message, ctx) -> Message:
        text_Message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_Message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
        

In [7]:
single_runtime = SingleThreadedAgentRuntime()
SimpleAgent.register(single_runtime, "SimpleAgent", lambda:SimpleAgent())
LLMAgent.register(single_runtime,"LLMAgent", lambda:LLMAgent())
single_runtime.start()

C:\Users\vishi\AppData\Local\Temp\ipykernel_9240\3198611198.py:2: RuntimeWarning: coroutine 'BaseAgent.register' was never awaited
  SimpleAgent.register(single_runtime, "SimpleAgent", lambda:SimpleAgent())
C:\Users\vishi\AppData\Local\Temp\ipykernel_9240\3198611198.py:3: RuntimeWarning: coroutine 'BaseAgent.register' was never awaited
  LLMAgent.register(single_runtime,"LLMAgent", lambda:LLMAgent())


### Distributed

In [ ]:
#%pip install --upgrade grpcio grpcio-tools
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost, GrpcWorkerAgentRuntime

host = GrpcWorkerAgentRuntimeHost(address="localhost:50052")
host.start()

In [ ]:
#now we will create 2 different RuntimeAgent to bind the agents
runtime1 = GrpcWorkerAgentRuntime(host_address="localhost:50052")
SimpleAgent.register(runtime1, "SimpleAgent", lambda:SimpleAgent())
runtime1.start()

runtime2 = GrpcWorkerAgentRuntime(host_address="localhost:50052")
LLMAgent.register(runtime2,"LLMAgent", lambda:LLMAgent())
runtime2.start()

##when we send the messages to this two agent they will be running separately on may be different machines